In [7]:
import dace

def vec_copy():
    sdfg = dace.SDFG("soft_hier_vec_copy")
    state = sdfg.add_state("main")
    a_host = sdfg.add_array("A", (256*32, ), dace.float16, dace.dtypes.StorageType.CPU_Heap, transient=True)
    a_dev = sdfg.add_array("soft_hier_A", (256*32, ), dace.float16, dace.dtypes.StorageType.SoftHier_HBM, transient=True)
    b_host = sdfg.add_array("B", (256*32, ), dace.float16, dace.dtypes.StorageType.CPU_Heap, transient=True)
    b_dev = sdfg.add_array("soft_hier_B", (256*32, ), dace.float16, dace.dtypes.StorageType.SoftHier_HBM, transient=True)
    ahc = state.add_access("A")
    adc = state.add_access("soft_hier_A")
    bhc = state.add_access("B")
    bdc = state.add_access("soft_hier_B")
    frag_a = sdfg.add_array("frag_A", (256, ), dace.float16, dace.dtypes.StorageType.SoftHier_TCDM, transient=True)
    frag_b = sdfg.add_array("frag_B", (256, ), dace.float16, dace.dtypes.StorageType.SoftHier_TCDM, transient=True)
    #af = state.add_access("frag_A")
    #bf = state.add_access("frag_B")

    dev_entry, dev_exit = state.add_map(name="copy_map_outer", ndrange={"i": dace.subsets.Range([(0, 256*32-1, 256*32)])}, schedule=dace.dtypes.ScheduleType.SoftHier_Device)
    tblock_entry, tblock_exit = state.add_map(name="copy_map_inner", ndrange={"ii": dace.subsets.Range([(0, 256*32-1, 256)])}, schedule=dace.dtypes.ScheduleType.SoftHier_Cluster)

    #glb_to_vecin = GlobalToVECIN(name="glb_to_vecin_a", input_names=["IN_A"], output_names=["OUT_frag_A"], queue_length=1, load_length=256)
    glb_to_vecin = state.add_access("frag_A")
    #vecout_to_glb = VECOUTToGlobal(name="vecout_to_glb_b", input_names=["IN_frag_B"], output_names=["OUT_B"], queue_length=1, load_length=256)
    libnode = state.add_access("frag_B")
    #libnode = VecUnit(name="vecout_to_glb_b", input_names=["IN_frag_A"], output_names=["OUT_frag_B"], queue_length=1, load_length=256)

    state.add_edge(ahc, None, adc, None, dace.memlet.Memlet(f"A[0:{256*32}]"))
    state.add_edge(adc, None, dev_entry, "IN_A", dace.memlet.Memlet(f"soft_hier_A[0:{256*32}]"))
    state.add_edge(dev_entry, "OUT_A", tblock_entry, "IN_A", dace.memlet.Memlet(f"soft_hier_A[0:{256*32}]"))
    state.add_edge(tblock_entry, "OUT_A", glb_to_vecin, None, dace.memlet.Memlet(f"soft_hier_A[i + ii:i + ii + 256]"))
    state.add_edge(glb_to_vecin, None, libnode, None, dace.memlet.Memlet(f"frag_A[0:256]"))
    state.add_edge(libnode, None, tblock_exit, "IN_B", dace.memlet.Memlet(f"soft_hier_B[i + ii:i + ii + 256]"))
    state.add_edge(tblock_exit, "OUT_B", dev_exit, "IN_B", dace.memlet.Memlet(f"soft_hier_B[i + ii:i + ii + 256]"))
    state.add_edge(dev_exit, "OUT_B", bdc, None, dace.memlet.Memlet(f"soft_hier_B[0:{256*32}]"))
    state.add_edge(bdc, None, bhc, None, dace.memlet.Memlet(f"B[0:{256*32}]"))

    for n in [dev_entry, tblock_entry]:
        n.add_in_connector("IN_A")
        n.add_out_connector("OUT_A")

    for n in [dev_exit, tblock_exit]:
        n.add_in_connector("IN_B")
        n.add_out_connector("OUT_B")

    #libnode.add_in_connector("IN_A")
    #libnode.add_out_connector("OUT_B")

    #t = state.add_tasklet(name="assign", inputes={"_in"}, outputs={"_out"}, code="_out = _in")


    sdfg.save("soft_hier_2.sdfgz")
    return sdfg

In [ ]:
s = vec_copy()

/scratch/dace4softhier/dace_soft_hier/dace/sdfg/validation.py:519: UserWarning: WARNING: Use of uninitialized transient "A" in state "main"
  warnings.warn('WARNING: Use of uninitialized transient "%s" in state "%s"' %
/scratch/dace4softhier/dace_soft_hier/dace/codegen/targets/soft_hier/soft_hier.py:1844: UserWarning: No `gpu_block_size` property specified on map "copy_map_outer". Falling back to the configuration entry `compiler.cuda.default_block_size`: 32,1,1. You can either specify the block size to use with the gpu_block_size property, or by adding nested `GPU_ThreadBlock` maps, which map work to individual threads. For more information, see https://spcldace.readthedocs.io/en/latest/optimization/gpu.html
  warnings.warn(


Using frame code generator for soft_hier_vec_copy
Allocated outer-level transients
Generating states
Generating states for SDFG soft_hier_vec_copy...
Using control flow detection...
Structured control flow tree
Reset block parents
Generating code for cft...
class GeneralBlock as_cpp
length of elements: 1
generate_transition: 0 for type <class 'dace.codegen.control_flow.BasicCFBlock'>
class BasicCFBlock as_cpp
state main has 10 nodes
Calling dispatch_state main...
Dispatching state main with DaCeCodeGenerator
Framecode generating state main...
Allocating arrays in scope for state main...
SoftHier: Allocating soft_hier_A
SoftHier: Allocating soft_hier_B
Finished allocating arrays in scope for state main.
len(components) <= 1: 1
~~~~~~~~~~~~~~~Dispatching subgraph main in state 0
***************Dispatching node A in state 0
11111111111
CPU: Generating node A in state 0
SoftHier: Emitting copy from A to soft_hier_A
***************Finished dispatching node A in state 0
***************Dispat

CompilationError: Compiler failure:
[35m[1mScanning dependencies of target dacestub_soft_hier_vec_copy[0m
[ 25%] [32mBuilding CXX object CMakeFiles/dacestub_soft_hier_vec_copy.dir/tools/dacestub.cpp.o[0m
[ 50%] [32m[1mLinking CXX shared library libdacestub_soft_hier_vec_copy.so[0m
[ 50%] Built target dacestub_soft_hier_vec_copy
[35m[1mScanning dependencies of target soft_hier_vec_copy[0m
[ 75%] [32mBuilding CXX object CMakeFiles/soft_hier_vec_copy.dir/scratch/dace4softhier/dace_soft_hier/dace/soft_hier/.dacecache/soft_hier_vec_copy/src/cpu/soft_hier_vec_copy.cpp.o[0m
In file included from /scratch/dace4softhier/dace_soft_hier/dace/codegen/../runtime/include/dace/dace.h:14,
                 from /scratch/dace4softhier/dace_soft_hier/dace/soft_hier/.dacecache/soft_hier_vec_copy/src/cpu/soft_hier_vec_copy.cpp:2:
/scratch/dace4softhier/dace_soft_hier/dace/codegen/../runtime/include/dace/types.h: In constructor ‘dace::half::half(float)’:
/scratch/dace4softhier/dace_soft_hier/da

In [3]:
from IPython.display import Code
Code(s.generate_code()[0].clean_code, language='cpp')

Using frame code generator for soft_hier_vec_copy
Allocated outer-level transients
Generating states
Generating states for SDFG soft_hier_vec_copy...
Using control flow detection...
Structured control flow tree
Reset block parents
Generating code for cft...
class GeneralBlock as_cpp
length of elements: 1
generate_transition: 0 for type <class 'dace.codegen.control_flow.BasicCFBlock'>
class BasicCFBlock as_cpp
state main has 10 nodes
Calling dispatch_state main...
Dispatching state main with DaCeCodeGenerator
Framecode generating state main...
Allocating arrays in scope for state main...
SoftHier: Allocating soft_hier_A
SoftHier: Allocating soft_hier_B
Finished allocating arrays in scope for state main.
len(components) <= 1: 1
~~~~~~~~~~~~~~~Dispatching subgraph main in state 0
***************Dispatching node A in state 0
11111111111
CPU: Generating node A in state 0
SoftHier: Emitting copy from A to soft_hier_A
***************Finished dispatching node A in state 0
***************Dispat

/scratch/dace4softhier/dace_soft_hier/dace/sdfg/validation.py:519: UserWarning: WARNING: Use of uninitialized transient "A" in state "main"
  warnings.warn('WARNING: Use of uninitialized transient "%s" in state "%s"' %
/scratch/dace4softhier/dace_soft_hier/dace/codegen/targets/soft_hier/soft_hier.py:1844: UserWarning: No `gpu_block_size` property specified on map "copy_map_outer". Falling back to the configuration entry `compiler.cuda.default_block_size`: 32,1,1. You can either specify the block size to use with the gpu_block_size property, or by adding nested `GPU_ThreadBlock` maps, which map work to individual threads. For more information, see https://spcldace.readthedocs.io/en/latest/optimization/gpu.html
  warnings.warn(


/* DaCe AUTO-GENERATED FILE. DO NOT MODIFY */
#include <dace/dace.h>
#include "../../include/hash.h"

struct soft_hier_vec_copy_state_t {
    dace::cuda::Context *gpu_context;
};

DACE_EXPORTED void __dace_runkernel_copy_map_outer_0_0_4(soft_hier_vec_copy_state_t *__state, const dace::float16 * __restrict__ soft_hier_A, dace::float16 * __restrict__ soft_hier_B);
void __program_soft_hier_vec_copy_internal(soft_hier_vec_copy_state_t*__state)
{

    {
        // Start allocate arrays in scope
        dace::float16 *A;
        A = new dace::float16 DACE_ALIGN(64)[8192];
        dace::float16 * soft_hier_A;
        DACE_ACL_CHECK(aclrtMalloc((void**)&soft_hier_A, 8192 * sizeof(dace::float16)));
        dace::float16 *B;
        B = new dace::float16 DACE_ALIGN(64)[8192];
        dace::float16 * soft_hier_B;
        DACE_ACL_CHECK(aclrtMalloc((void**)&soft_hier_B, 8192 * sizeof(dace::float16)));
        // Finish allocate arrays in scope
        // SoftHier: Emitting copy from A to soft_hier_A

        dace::CopyND<dace::float16, 1, false, 8192>::template ConstDst<1>::Copy(
        A, soft_hier_A, 1);
        __dace_runkernel_copy_map_outer_0_0_4(__state, soft_hier_A, soft_hier_B);
        // SoftHier: Emitting copy from soft_hier_B to B

        dace::CopyND<dace::float16, 1, false, 8192>::template ConstDst<1>::Copy(
        soft_hier_B, B, 1);
        delete[] A;
        delete[] B;

    }
}

DACE_EXPORTED void __program_soft_hier_vec_copy(soft_hier_vec_copy_state_t *__state)
{
    __program_soft_hier_vec_copy_internal(__state);
}
DACE_EXPORTED int __dace_init_soft_hier(soft_hier_vec_copy_state_t *__state);
DACE_EXPORTED int __dace_exit_soft_hier(soft_hier_vec_copy_state_t *__state);

DACE_EXPORTED soft_hier_vec_copy_state_t *__dace_init_soft_hier_vec_copy()
{
    int __result = 0;
    soft_hier_vec_copy_state_t *__state = new soft_hier_vec_copy_state_t;


    __result |= __dace_init_soft_hier(__state);

    if (__result) {
        delete __state;
        return nullptr;
    }
    return __state;
}

DACE_EXPORTED int __dace_exit_soft_hier_vec_copy(soft_hier_vec_copy_state_t *__state)
{
    int __err = 0;

    int __err_soft_hier = __dace_exit_soft_hier(__state);
    if (__err_soft_hier) {
        __err = __err_soft_hier;
    }
    delete __state;
    return __err;
}

In [4]:
Code(s.generate_code()[1].clean_code, language='cpp')

Using frame code generator for soft_hier_vec_copy
Allocated outer-level transients
Generating states
Generating states for SDFG soft_hier_vec_copy...
Using control flow detection...
Structured control flow tree
Reset block parents
Generating code for cft...
class GeneralBlock as_cpp
length of elements: 1
generate_transition: 0 for type <class 'dace.codegen.control_flow.BasicCFBlock'>
class BasicCFBlock as_cpp
state main has 10 nodes
Calling dispatch_state main...
Dispatching state main with DaCeCodeGenerator
Framecode generating state main...
Allocating arrays in scope for state main...
SoftHier: Allocating soft_hier_A
SoftHier: Allocating soft_hier_B
Finished allocating arrays in scope for state main.
len(components) <= 1: 1
~~~~~~~~~~~~~~~Dispatching subgraph main in state 0
***************Dispatching node A in state 0
11111111111
CPU: Generating node A in state 0
SoftHier: Emitting copy from A to soft_hier_A
***************Finished dispatching node A in state 0
***************Dispat

#include <cuda_runtime.h>
#include <dace/dace.h>


struct soft_hier_vec_copy_state_t {
    dace::cuda::Context *gpu_context;
};



DACE_EXPORTED int __dace_init_cuda(soft_hier_vec_copy_state_t *__state);
DACE_EXPORTED int __dace_exit_cuda(soft_hier_vec_copy_state_t *__state);



int __dace_init_cuda(soft_hier_vec_copy_state_t *__state) {
    int count;

    // Check that we are able to run cuda code
    if (cudaGetDeviceCount(&count) != cudaSuccess)
    {
        printf("ERROR: GPU drivers are not configured or cuda-capable device "
               "not found\n");
        return 1;
    }
    if (count == 0)
    {
        printf("ERROR: No cuda-capable devices found\n");
        return 2;
    }

    // Initialize cuda before we run the application
    float *dev_X;
    DACE_GPU_CHECK(cudaMalloc((void **) &dev_X, 1));
    DACE_GPU_CHECK(cudaFree(dev_X));

    

    __state->gpu_context = new dace::cuda::Context(1, 1);

    // Create cuda streams and events
    for(int i = 0; i < 1; ++i) {
        DACE_GPU_CHECK(cudaStreamCreateWithFlags(&__state->gpu_context->internal_streams[i], cudaStreamNonBlocking));
        __state->gpu_context->streams[i] = __state->gpu_context->internal_streams[i]; // Allow for externals to modify streams
    }
    for(int i = 0; i < 1; ++i) {
        DACE_GPU_CHECK(cudaEventCreateWithFlags(&__state->gpu_context->events[i], cudaEventDisableTiming));
    }

    

    return 0;
}

int __dace_exit_cuda(soft_hier_vec_copy_state_t *__state) {
    

    // Synchronize and check for CUDA errors
    int __err = static_cast<int>(__state->gpu_context->lasterror);
    if (__err == 0)
        __err = static_cast<int>(cudaDeviceSynchronize());

    // Destroy cuda streams and events
    for(int i = 0; i < 1; ++i) {
        DACE_GPU_CHECK(cudaStreamDestroy(__state->gpu_context->internal_streams[i]));
    }
    for(int i = 0; i < 1; ++i) {
        DACE_GPU_CHECK(cudaEventDestroy(__state->gpu_context->events[i]));
    }

    delete __state->gpu_context;
    return __err;
}

DACE_EXPORTED bool __dace_gpu_set_stream(soft_hier_vec_copy_state_t *__state, int streamid, gpuStream_t stream)
{
    if (streamid < 0 || streamid >= 1)
        return false;

    __state->gpu_context->streams[streamid] = stream;

    return true;
}

DACE_EXPORTED void __dace_gpu_set_all_streams(soft_hier_vec_copy_state_t *__state, gpuStream_t stream)
{
    for (int i = 0; i < 1; ++i)
        __state->gpu_context->streams[i] = stream;
}

__global__ void __launch_bounds__(32) copy_map_outer_0_0_4(const dace::float16 * __restrict__ soft_hier_A, dace::float16 * __restrict__ soft_hier_B) {
    {
        // TEST KERNEL SCOPE
        int i = (8192 * (blockIdx.x * 32 + threadIdx.x));
        if (i < 8192) {
            {
                // TEST DEVICE SCOPE
                {
                    dace::float16 * frag_A;
                    DACE_ACL_CHECK(aclrtMalloc((void**)&frag_A, 256 * sizeof(dace::float16)));
                    dace::float16 * frag_B;
                    DACE_ACL_CHECK(aclrtMalloc((void**)&frag_B, 256 * sizeof(dace::float16)));
                    // SoftHier: Emitting copy from soft_hier_A to frag_A

                    dace::CopyND<dace::float16, 1, false, 256>::template ConstDst<1>::Copy(
                    soft_hier_A + (i + ii), frag_A, 1);
                    // SoftHier: Emitting copy from frag_A to frag_B

                    dace::CopyND<dace::float16, 1, false, 256>::template ConstDst<1>::Copy(
                    frag_A, frag_B, 1);
                    // SoftHier: Emitting copy from frag_B to soft_hier_B

                    dace::CopyND<dace::float16, 1, false, 256>::template ConstDst<1>::Copy(
                    frag_B, soft_hier_B + (i + ii), 1);
                }
            }
        }
    }
}


DACE_EXPORTED void __dace_runkernel_copy_map_outer_0_0_4(soft_hier_vec_copy_state_t *__state, const dace::float16 * __restrict__ soft_hier_A, dace::float16 * __restrict__ soft_hier_B);
void __dace_runkernel_copy_map_outer_0_0_4(s

In [5]:
Code(s.generate_code()[2].clean_code, language='cpp')

Using frame code generator for soft_hier_vec_copy
Allocated outer-level transients
Generating states
Generating states for SDFG soft_hier_vec_copy...
Using control flow detection...
Structured control flow tree
Reset block parents
Generating code for cft...
class GeneralBlock as_cpp
length of elements: 1
generate_transition: 0 for type <class 'dace.codegen.control_flow.BasicCFBlock'>
class BasicCFBlock as_cpp
state main has 10 nodes
Calling dispatch_state main...
Dispatching state main with DaCeCodeGenerator
Framecode generating state main...
Allocating arrays in scope for state main...
SoftHier: Allocating soft_hier_A
SoftHier: Allocating soft_hier_B
Finished allocating arrays in scope for state main.
len(components) <= 1: 1
~~~~~~~~~~~~~~~Dispatching subgraph main in state 0
***************Dispatching node A in state 0
11111111111
CPU: Generating node A in state 0
SoftHier: Emitting copy from A to soft_hier_A
***************Finished dispatching node A in state 0
***************Dispat

#include <dace/dace.h>
typedef void * soft_hier_vec_copyHandle_t;
extern "C" soft_hier_vec_copyHandle_t __dace_init_soft_hier_vec_copy();
extern "C" int __dace_exit_soft_hier_vec_copy(soft_hier_vec_copyHandle_t handle);
extern "C" void __program_soft_hier_vec_copy(soft_hier_vec_copyHandle_t handle);

In [ ]:
Code(s.generate_code()[3].clean_code, language='cpp')
# var

Using frame code generator for soft_hier_vec_copy
Allocated outer-level transients
Generating states
Generating states for SDFG soft_hier_vec_copy...
Using control flow detection...
Structured control flow tree
Reset block parents
Generating code for cft...
class GeneralBlock as_cpp
length of elements: 1
generate_transition: 0 for type <class 'dace.codegen.control_flow.BasicCFBlock'>
class BasicCFBlock as_cpp
state main has 10 nodes
Calling dispatch_state main...
Dispatching state main with DaCeCodeGenerator
Framecode generating state main...
Allocating arrays in scope for state main...
SoftHier: Allocating soft_hier_A
SoftHier: Allocating soft_hier_B
Finished allocating arrays in scope for state main.
len(components) <= 1: 1
~~~~~~~~~~~~~~~Dispatching subgraph main in state 0
***************Dispatching node A in state 0
11111111111
CPU: Generating node A in state 0
SoftHier: Emitting copy from A to soft_hier_A
***************Finished dispatching node A in state 0
***************Dispat

#include <cstdlib>
#include "../include/soft_hier_vec_copy.h"

int main(int argc, char **argv) {
    soft_hier_vec_copyHandle_t handle;
    int err;


    handle = __dace_init_soft_hier_vec_copy();
    __program_soft_hier_vec_copy(handle);
    err = __dace_exit_soft_hier_vec_copy(handle);



    return err;
}